# Build a Prompt Chaining Agentic Workflow Using Langgraph

In this lab, we will build a chained prompts workflow using Langgraph Agentic Framework to intelligently process a legal contract, draft an email with concerns in the contract, and finally provide feedback on the email. Chaining prompts, which involves using the output of one prompt as input to another, offers several advantages when working with Large Language Models (LLMs).

## What is Prompt Chaining
Prompt chaining is a technique that involves breaking down a workflow into a series of known steps. Then orchestrates each step (typically involve an LLM invocation) to handle the output generated from the previous step. Within the worklow, there could be one or more conditional steps which determines the the trajectory of the workflow based on the given state.

## Architecture Diagram
<img src="../../imgs/prompt-chaining-architecture.png" width=800>

Let's get started!

## Import Dependencies and Configure Logging
Set up logging configuration and import required libraries for the prompt chaining workflow.

In [ ]:
import json
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

## AWS Services Initialization
Initialize AWS clients and services including Bedrock, S3, and configure regional settings.

In [ ]:
import boto3
from langchain_aws import ChatBedrockConverse
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)
default_bucket_name = f"labs-bucket-{region}-{account_id}" # replace the bucket name if running outside of the AWS facilitated environment.

Define a helper function to retrieve contract documents from S3 for processing.

In [ ]:
import boto3
from botocore.exceptions import ClientError

def retrieve_from_s3(bucket_name, contract_name):
    """
    Retrieve a contract document from S3
    
    Args:
        bucket_name (str): Name of the S3 bucket
        contract_name (str): Key/filename of the contract in S3
    
    Returns:
        str: Content of the contract document
    """
    try:
        # Create S3 client
        s3_client = boto3.client('s3')
        
        # Get the object from S3
        response = s3_client.get_object(Bucket=bucket_name, Key=contract_name)
        
        # Read and decode the content
        contract_content = response['Body'].read().decode('utf-8')
        
        return contract_content
        
    except ClientError as e:
        logger.error(f"Error retrieving contract from S3: {e}")
        return None
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        return None

## LLM Setup and State Definition
Configure the Language Model and define the state structure for the workflow.

In [ ]:
model_id = "us.amazon.nova-micro-v1:0"
llm = ChatBedrockConverse(
    model=model_id,  # or another Claude model
    temperature=0.3,
    max_tokens=None,
    client=bedrock_client,
)

# Graph state
class State(TypedDict):
    s3_bucket_name : str
    s3_object_key: str
    email_concerns: str
    email_content: str
    email_review_feedback: str
    rewritten_email: str

## Step 1: Concern Finder Function
Define the first step in the chain that analyzes the contract and identifies legal risks and concerns.

In [ ]:
def concern_finder(state: State):
    system_prompt = """You’re our Chief Legal Officer. You are given a contract to review for risks, focusing on data privacy, SLAs, and liability caps.

Output your findings in <risks> XML tags.
"""

    contract_s3_key = state["s3_object_key"]
    bucket_name = state["s3_bucket_name"]
    contract_data = retrieve_from_s3(bucket_name, contract_s3_key)
    formatted_contract_data = f"<contract>\n{contract_data}\n</contract>"
    messages = [ ("system", system_prompt), ("human", formatted_contract_data)]

    email_concerns = llm.invoke(messages)
    return {"email_concerns": email_concerns.content}

## Step 2: Email Writer Function
Define the second step that drafts a professional email based on the identified contract concerns.

In [ ]:
def email_writer(state: State):
    system_prompt = """You are an expert at writing corporate legal emails. You are given the concerns from a contract in <risk> XML tag. Your task is to draft an email to Frost Technologies outlining the following concerns and proposing changes to a contract.
"""
    messages = [ ("system", system_prompt), ("human", state["email_concerns"])]
    email_content = llm.invoke(messages)
    return {"email_content": email_content.content}

## Step 3: Email Reviewer Function
Define the third step that reviews the drafted email and provides feedback on tone, clarity, and professionalism.

In [ ]:
def email_reviewer(state: State):
    system_prompt = """You are a professional email reviewer. You are given an email content in <email> XML tag. Your task is to review an email and provide feedback. 
Give feedback on tone, clarity, and professionalism.
"""
    messages = [ ("system", system_prompt), ("human", f"<email>{state['email_content']}</email>")]
    email_review_feedback = llm.invoke(messages)
    return {"email_review_feedback": email_review_feedback.content}

## Step 4: Email Rewriter Function
Define the final step that incorporates the reviewer feedback and rewrites the email for improved quality.

In [ ]:
def email_rewriter(state: State):
    system_prompt = """You are a professional email writer. You are given an original email content in <email> XML tag, and the feedback from a reviewer in <feedback> XML tag. Your task is to incorporate the feedback and rewrite the email from the orignial email.
Put your rewritten email in the <rewritten_email> XML tag. Do not provide any explainations.
"""
    messages = [ ("system", system_prompt), ("human", f"<email>{state['email_content']}</email>\n<feedback>{state['email_review_feedback']}</feedback>")]
    rewritten_email = llm.invoke(messages)
    return {"rewritten_email": rewritten_email.content}


## Workflow Construction and Execution
Build the LangGraph workflow by connecting all the functions in sequence and execute the prompt chaining pipeline.

In [ ]:
# Build workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("concern_finder", concern_finder)
workflow.add_node("email_writer", email_writer)
workflow.add_node("email_reviewer", email_reviewer)
workflow.add_node("email_rewriter", email_rewriter)
# workflow.add_node("improve_joke", improve_joke)
# workflow.add_node("polish_joke", polish_joke)

# Add edges to connect nodes
workflow.add_edge(START, "concern_finder")
workflow.add_edge("concern_finder", "email_writer")
workflow.add_edge("email_writer", "email_reviewer")
workflow.add_edge("email_reviewer", "email_rewriter")
workflow.add_edge("email_rewriter", END)

# Compile
chain = workflow.compile()

# Show workflow
display(Image(chain.get_graph().draw_mermaid_png()))

## Testing the Workflow

In [ ]:
test_input_data = {
    "s3_object_key" : "data/agents/contract.txt",
    "s3_bucket_name" : default_bucket_name
    
}
# Invoke
state = chain.invoke(test_input_data)

Print out the output for each step in the workflow

In [ ]:
print(f"concern finder results: {state['email_concerns']}")

In [ ]:
print(f"email writer results: {state['email_content']}")

In [ ]:
print(f"email reviewer feedback results: {state['email_review_feedback']}")

In [ ]:
print(f"email rewriter results: {state['rewritten_email']}")

## Summary
This notebook demonstrates how to build an intelligent prompt chaining workflow using the LangGraph framework to automate legal contract analysis and professional email generation. The workflow processes a legal contract stored in S3, identifies potential risks and concerns, drafts a professional email outlining these issues, reviews the email for quality, and produces a refined final version.
